Gold Layer: Create an aggregated view with the quantity of breweries per
type and location

In [0]:
spark.conf.set('spark.sql.adaptive.enabled', 'true')
spark.conf.set('spark.databricks.delta.optimizeWrite.enabled', 'true')

In [0]:
from pyspark.sql.functions import col, concat_ws, count

In [0]:
silver_table = 'ws_bees_dev_.db_silver_dev.tb_openbrewery_api'
silver_checkpoint = 'abfss://ctr-sl-dev@stgbeesdev.dfs.core.windows.net/_checkopints/openbrewery/api/'

gold_path = 'abfss://ctr-gd-dev@stgbeesdev.dfs.core.windows.net/openbrewery/api/'
gold_table = 'ws_bees_dev_.db_gold_dev.tb_openbrewery_api'

In [0]:
openbrewery_df = (
    spark.readStream
    .format('delta').table(silver_table)
    .groupBy(
        concat_ws(', ', 'state', 'country').alias('location'),
        col('brewery_type').alias('type'))
    .agg(count('*').alias('quantity')))

In [0]:
query = (
    openbrewery_df
    .writeStream.format('delta')
    .outputMode("complete")
    .trigger(availableNow=True)
    .option("checkpointLocation", silver_checkpoint)
    .option('path', gold_path)
    .toTable(gold_table))

query.awaitTermination()

In [0]:
_ = spark.sql('OPTIMIZE '+gold_table) 